In [37]:
import numpy as np
import QuantumCompilerV2 as QC2
from RB_tool import *
# from qel.drivers.Tektronix.Tektronix_AWG import AWG5208

flat_list = np.array([35.7,34.3,12.4,12])*1e-9 #I_pi, Q_pi, I_pi/2, Q_pi/2
G = Gate_1q(flat_list,1e-9*10,3)

In [38]:
import random

#randomized list
seq_truncated_list = [1,2,3,5,7,10,13]
randomlist = [random.choices(range(len(G.gatelist)-1),k = idx) for idx in seq_truncated_list]
print(randomlist)


[[7], [6, 0], [2, 6, 7], [10, 2, 6, 2, 10], [8, 10, 5, 3, 1, 3, 11], [8, 5, 8, 11, 5, 11, 9, 7, 9, 4], [4, 11, 6, 10, 10, 3, 3, 10, 9, 0, 2, 0, 2]]


Process

In [39]:
#find reverse
result_list = []
ground = np.array([0,0,NaN])
for i,seq in enumerate(randomlist):
    state = ground
    print(state)
    result_list.append([])
    for id in seq:
        state = G.operator(id,state)
        result_list[i].append(G.gatelist[id])
    result_list[i].append(G.gatelist[G.reverse(state)])

print(result_list)

#virtual Z
eff_list = []
for i,seq in enumerate(result_list):
    eff_list.append(Virtual_Z(seq))

print(eff_list)
# print(len(eff_list))


[ 0.  0. nan]
[ 0.  1. nan]
[ 1.  1. nan]
[ 2.  1. nan]
[ 2.  0. nan]
[ 2.  3. nan]
[ 1.  3. nan]
[['Y2', '-Y2'], ['X2', 'X', 'X2'], ['Z', 'X2', 'Y2', 'Y2'], ['-Y2', 'Z', 'X2', 'Z', '-Y2', 'X2'], ['Z2', '-Y2', '-Z', '-X', 'Y', '-X', '-Z2', 'X2'], ['Z2', '-Z', 'Z2', '-Z2', '-Z', '-Z2', '-X2', 'Y2', '-X2', '-Y', 'X'], ['-Y', '-Z2', 'X2', '-Y2', '-Y2', '-X', '-X', '-Y2', '-X2', 'X', 'Z', 'X', 'Z', 'Y2']]
[['Y2', '-Y2'], ['X2', 'X', 'X2'], ['X2', 'Y2', 'Y2'], ['Y2', '-X2', '-Y2', 'X2'], ['Y2', '-X', 'Y', 'Y', 'X2'], ['-X2', 'Y2', '-X2', '-Y', 'X'], ['-X', 'X2', '-Y2', '-Y2', '-X', '-X', '-Y2', '-X2', '-X', '-X', 'Y2']]


In [7]:
channels = [
    'ch1_I',
    'ch2_Readout',
    'ch1_Q',
]

RB_Sequence = {
    channels[0]: [None for i in range(len(seq_truncated_list))],
    channels[1]: [None for i in range(len(seq_truncated_list))],
    channels[2]: [None for i in range(len(seq_truncated_list))],
    'marker'   : [None for i in range(len(seq_truncated_list))]
}

#Compile

for i,seq in enumerate(eff_list):
    gate = Gate(name= 'main')
    gate['Q1'] = Bus.null((2,1e-9))
    gate['readout'] = Bus.null((2,1e-9))

    for sym in seq:
        if sym !='I':
            idx = G.perform_gate_list.index(sym)
            gate += G.gate_seq[idx]
    gate += G.gate_seq[-1]
    Q1 = gate.compile_device('Q1')
    Q1.wires = {0: 'I', 1: 'Q'}
    channel_readout = gate.compile_device('readout')
    channel_readout.wires = {0: 'readout', 1: 'marker'}

    RB_Sequence[channels[0]][i] = Q1.get('I')
    RB_Sequence[channels[1]][i] = channel_readout.get('readout')
    RB_Sequence[channels[2]][i] = Q1.get('Q')
    RB_Sequence['marker'][i] = channel_readout.get('marker')

maxlength = max([RB_Sequence[channels[1]][i].span for i in range(len(seq_truncated_list))])
for i,seq in enumerate(eff_list):
    RB_Sequence[channels[0]][i] = RB_Sequence[channels[0]][i].padConst((maxlength - RB_Sequence[channels[0]][i].span,0),(0,0))
    RB_Sequence[channels[1]][i] = RB_Sequence[channels[1]][i].padConst((maxlength - RB_Sequence[channels[1]][i].span,0),(0,0))
    RB_Sequence[channels[2]][i] = RB_Sequence[channels[2]][i].padConst((maxlength - RB_Sequence[channels[2]][i].span,0),(0,0))
    RB_Sequence['marker'][i] = RB_Sequence['marker'][i].padConst((maxlength - RB_Sequence['marker'][i].span,0),(0,0))

    RB_Sequence[channels[0]][i] = RB_Sequence[channels[0]][i].output
    RB_Sequence[channels[1]][i] = RB_Sequence[channels[1]][i].output
    RB_Sequence[channels[2]][i] = RB_Sequence[channels[2]][i].output
    RB_Sequence['marker'][i] = RB_Sequence['marker'][i].output


In [ ]:
# connect to AWG
awg = AWG5208(
    inst_name='AWG5208',
    inst_address='TCPIP0::192.168.20.43::inst0::INSTR')
# set sampling rate and reference clock
awg.set_sample_rate(sample_rate=1e9)
awg.set_extref_source(ref_freq=10e6)

In [ ]:
# stop and clear sequence list and waveform list in AWG.
awg.clean_up()

In [ ]:
# create waveform and upload to AWG
for idx in range(len(seq_truncated_list)):
    awg.set_wfm(
        wfm_name=f'{channels[0]}_{idx}',
        wfm=RB_Sequence[channels[0]][idx])
    awg.set_wfm(
        wfm_name=f'{channels[1]}_{idx}',
        wfm=RB_Sequence[channels[1]][idx],
        mkr1=RB_Sequence['marker'][idx])
    awg.set_wfm(
        wfm_name=f'{channels[2]}_{idx}',
        wfm=RB_Sequence[channels[2]][idx])
awg.upload_wfm()

In [ ]:
seq_name = 'RB_Sequence'
# assign waveform to sequence table
awg.set_seq(seq_name, num_track=len(channels), num_step=len(seq_truncated_list))

for track_idx, track in enumerate(channels):
    for step_idx, time in enumerate(seq_truncated_list):
        awg.assign_seq(
            f"{track}_{step_idx}", seq_name, track_idx+1, step_idx+1)
awg.upload_seq(seq_name)

In [ ]:
# assign sequences to channels
awg.assign_ch(1, seq_name, track_index=1)
awg.assign_ch(2, seq_name, track_index=2)
awg.assign_ch(3, seq_name, track_index=3)

# set channel amplitudes
awg.set_ch_amp(ch=1, wfm_Vpp=1.0, wfm_offset=0., mkr1=1.0, mkr2=1.0, mkr3=1.0, mkr4=1.0)
awg.set_ch_amp(ch=2, wfm_Vpp=1.0, wfm_offset=0., mkr1=1.0, mkr2=1.0, mkr3=1.0, mkr4=1.0)
awg.set_ch_amp(ch=3, wfm_Vpp=1.0, wfm_offset=0., mkr1=1.0, mkr2=1.0, mkr3=1.0, mkr4=1.0)

In [36]:
import numpy as np
from numpy import NaN
state = np.array([2,2,NaN])

temp = np.array([0,0,NaN]) - state
axis = np.where(~np.isnan(temp) == True)[0][0]
R_state = int(temp[axis])%4
idx = 12 if R_state == 0 else axis + (R_state%2)*6 + (R_state%3 == 0)*3
print(idx)

0
